![Portada](Portada.png)

In [3]:
import pandas as pd
import sqlite3

In [3]:
#Conexión a la base de datos
connection = sqlite3.connect("/Users/alvarosanchez/ONLINE_DS_THEBRIDGE_ALVAROSMMS-1/ONLINE_DS_THEBRIDGE_ALVAROSMMS/Sprint_6/Team_Challenge_SQL/sql-murder-mystery.db")

In [4]:
#Se optine el cursor
game_cursor = connection.cursor()

In [5]:
#Definición la funcion para leer datos y pasar a un dataframe de pandas

def sql_query(query):
    game_cursor.execute(query)
    ans = game_cursor.fetchall()
    names = [description[0] for description in game_cursor.description]
    #Modifico la funcion para poder ver las columnas enteras 
    for row in ans:
        print([str(col) for col in row])  # Convierte todas las columnas a string para evitar truncamientos
    
    return pd.DataFrame(ans, columns=names)


In [6]:
#Query para ver los nombres de las tablas
query = ''' 
SELECT name 
FROM sqlite_master
where type = 'table'
'''
sql_query(query)

['crime_scene_report']
['drivers_license']
['person']
['facebook_event_checkin']
['interview']
['get_fit_now_member']
['get_fit_now_check_in']
['income']
['solution']


,name
0,crime_scene_report
1,drivers_license
2,person
3,facebook_event_checkin
4,interview
5,get_fit_now_member
6,get_fit_now_check_in
7,income
8,solution


![Diagrama](Diagrama.png)

In [8]:
# Actualizo la columna 'date' al formato europeo
game_cursor.execute("""
    UPDATE crime_scene_report
    SET date = SUBSTR(date, 7, 2) || '-' || SUBSTR(date, 5, 2) || '-' || SUBSTR(date, 1, 4)
""")

![Pistas](pistas.png)

In [9]:
#Filtramos por type: murder, date: 15-01-2018, city =SQL City para obtener el informe.
query='''
SELECT *
FROM crime_scene_report
WHERE type = "murder"
	AND date = "15-01-2018"
	AND city = "SQL City";
'''
sql_query(query)

['15-01-2018', 'murder', 'Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".', 'SQL City']


,date,type,description,city
0,15-01-2018,murder,Security footage shows that there were 2 witne...,SQL City


![Informe](Informe.png)

In [10]:
#Identificamos el primer testigo
query='''
SELECT *
FROM person
WHERE address_street_name = "Northwestern Dr"
ORDER BY address_number DESC
LIMIT 1;
'''
sql_query(query)

['14887', 'Morty Schapiro', '118009', '4919', 'Northwestern Dr', '111564949']


,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [11]:
#Identificamos a la segunda testigo
query='''
SELECT *
FROM person
WHERE name LIKE '%Annabel%'
AND address_street_name = "Franklin Ave";
'''
sql_query(query)

['16371', 'Annabel Miller', '490173', '103', 'Franklin Ave', '318771143']


,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [ ]:
#Con los IDs de cada uno sacamos los informes de sus entrevistas.
query='''
SELECT *
FROM interview
WHERE person_id IN ("14887", "16371");
'''
sql_query(query)

['14887', 'I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".']
['16371', 'I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.']


,person_id,transcript
0,14887,I heard a gunshot and then saw a man run out. ...
1,16371,"I saw the murder happen, and I recognized the ..."


![Nuevas pistas](Nuevaspistas.png)

In [12]:
#Con las nuevas pistas, cruzamos las tablas "get_fit_now_member","person" y "divers_license" con un DOBLE JOIN
query=''' 
SELECT person.id, person.name
FROM get_fit_now_member
JOIN person
ON get_fit_now_member.person_id = person.id
JOIN drivers_license
ON person.license_id = drivers_license.id
WHERE get_fit_now_member.id
LIKE "48Z%"
AND get_fit_now_member.membership_status = "gold"
AND drivers_license.plate_number LIKE "%H42W%"
'''
sql_query(query)

['67318', 'Jeremy Bowers']


,id,name
0,67318,Jeremy Bowers


![Asesino](Asesino.png)

In [13]:
#Volvemos a la tabla "interviews" para ver el testimonio.
query=''' 
SELECT *
FROM interview
WHERE person_id=67318
'''
sql_query(query)

['67318', 'I was hired by a woman with a lot of money. I don\'t know her name but I know she\'s around 5\'5" (65") or 5\'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n']


,person_id,transcript
0,67318,I was hired by a woman with a lot of money. I ...


![Informe_Asesino](Informeasesino.png)

In [14]:
#Con estos datos, vamos a cruzar las tablas "person", "drivers_license" y "facebook_event_checkin",y encontrar a la cabeza pensante de este asesinato.
query= ''' 
SELECT fe.person_id, dl.*, p.name
FROM (
    SELECT person_id
    FROM facebook_event_checkin
    WHERE event_name = 'SQL Symphony Concert' AND date BETWEEN 20171201 AND 20171231
    GROUP BY person_id
    HAVING COUNT(*) = 3
) AS fe
JOIN person AS p
ON fe.person_id = p.id
JOIN drivers_license AS dl
ON p.license_id = dl.id
WHERE dl.car_make = 'Tesla' AND dl.car_model = 'Model S'
AND dl.height BETWEEN 65 AND 67;
'''
sql_query(query)

['99716', '202298', '68', '66', 'green', 'red', 'female', '500123', 'Tesla', 'Model S', 'Miranda Priestly']


,person_id,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model,name
0,99716,202298,68,66,green,red,female,500123,Tesla,Model S,Miranda Priestly


![fin](Final.jpg)

In [ ]:
connection.commit()